In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from order_of_magnitude import order_of_magnitude
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from itertools import combinations

from scipy.stats import mannwhitneyu, wilcoxon, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2, chi2_contingency, chisquare

from fitter import Fitter, get_common_distributions, get_distributions
from scipy.stats import kstest, ks_2samp

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score
import statsmodels.api as sm

from sklearn.utils import resample

In [58]:
data = pd.read_excel('Exercise.xlsx')
data.columns = list(data.iloc[0, :].values)
data.drop(index=0, inplace=True)
data.drop('Name', axis=1, inplace=True)
data.head()

C:\Users\Akshit Tayade\anaconda3\envs\akshit_venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Body Type,Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE),Suggestion
1,Male,18,152,50,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),2067.152431,Loose Weight,80,21.641274,1392.866,Boy,17.878324,Normal,1671.4392,2131.08498,-0.03,LOW CARB DIET (25:35:40)
2,Male,18,152,54,1. Sedentary or light activity (e.g. - Office ...,2. Lightly active (light exercise/sports 1-3 d...,1705.730935,Maintain Weight,78,23.372576,1447.866,Boy,20.49259,Normal,1990.81575,2215.23498,-0.23,BALANCE DIET (55:25:20)
3,Male,18,152,58,1. Sedentary or light activity (e.g. - Office ...,3. Moderately active (moderate exercise/sports...,1910.142686,Gain Muscle,65,25.103878,1502.866,Boy,23.106856,Overweight,2329.4423,2299.38498,-0.18,BALANCE DIET (55:25:20)
4,Male,18,152,62,3. Vigorously active (Agricultural worker (non...,4. Very active (hard exercise/sports 6-7 days ...,2552.952908,Loose Weight,82,26.83518,1557.866,Boy,25.721122,Overweight,2687.31885,3505.1985,-0.271667,KETOGENIC DIET (5:35:60)
5,Male,18,152,66,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),1826.086885,Maintain Weight,75,28.566482,1612.866,Boy,28.335388,Overweight,1935.4392,2467.68498,-0.26,BALANCE DIET (55:25:20)


In [59]:
# Mapping String to numeric values.
data['Gender']=data['Gender'].map({"Male":0,"Female":1})
data['Body Type']=data['Body Type'].map({"Underweight":0,"Normal":1,"Overweight":2,"Obese":3})
data['Type']=data['Type'].map({"Boy":0,"Girl":1,"Adult Male":2,"Adult Female":3})
data['Suggestion']=data['Suggestion'].map({"LOW CARB DIET (25:35:40)":0,"BALANCE DIET (55:25:20)":1,"ZONE DIET (40:30:30)":2,"KETOGENIC DIET (5:35:60)":3,"DEPLETION DIET (DYNAMIC)":4,"HIGH CARB DIET(60:20:20)":5})
data['Target']=data['Target'].map({"Loose Weight":0,"Maintain Weight":1,"Gain Muscle":2})
data['Lifestyle']=data['Lifestyle'].map({"1. Sedentary or light activity (e.g. - Office worker getting little or no exercise)":0,"2. Active or moderately active (Construction worker or person running one hour daily)":1,"3. Vigorously active (Agricultural worker (non mechanized) or person swimming two hours daily)":2})
data['Exercise']=data['Exercise'].map({"1. Sedentary (little or no exercise)":0,"2. Lightly active (light exercise/sports 1-3 days/ week)":1,"3. Moderately active (moderate exercise/sports 3-5days/week)":2,"4. Very active (hard exercise/sports 6-7 days a week)":3})

data = data.apply(pd.to_numeric)

In [60]:
data.drop('Body Type', axis=1, inplace=True)

In [61]:
sc_scaler = StandardScaler()
X_ = sc_scaler.fit_transform(data.iloc[:, :-1])
X_ = pd.DataFrame(X_, columns=list(data.columns)[:-1])
X_

,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE)
0,-1.0,-1.704309,-1.46385,-1.243933,-1.224988,-1.360019,-0.948266,-1.224745,0.237504,-0.592030,-0.541999,-3.618136,-1.298428,-1.199687,-1.055461,-0.168609
1,-1.0,-1.704309,-1.46385,-1.006993,-1.224988,-0.473050,-1.385131,0.000000,0.043670,-0.342490,-0.321641,-3.618136,-1.017369,-0.530306,-0.926927,-1.307818
2,-1.0,-1.704309,-1.46385,-0.770054,-1.224988,0.413919,-1.138050,1.224745,-1.216250,-0.092950,-0.101283,-3.618136,-0.736309,0.179420,-0.798394,-1.023016
3,-1.0,-1.704309,-1.46385,-0.533114,1.224502,1.300887,-0.361060,-1.224745,0.431338,0.156589,0.119074,-3.618136,-0.455250,0.929492,1.043408,-1.545153
4,-1.0,-1.704309,-1.46385,-0.296174,-1.224988,-1.360019,-1.239652,0.000000,-0.247081,0.406129,0.339432,-3.618136,-0.174191,-0.646370,-0.541326,-1.478699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,1.0,1.565182,1.46385,0.296174,1.224502,1.300887,1.246541,0.000000,1.109757,-0.404250,-0.455130,0.904534,0.766252,0.411337,0.550863,1.255402
10076,1.0,1.565182,1.46385,0.533114,1.224502,-1.360019,1.136394,1.224745,1.303591,-0.230196,-0.301873,0.904534,0.922045,-1.048948,0.682325,0.913640
10077,1.0,1.565182,1.46385,0.770054,1.224502,-0.473050,-0.283890,-1.224745,-0.731665,-0.056142,-0.148616,0.904534,1.077837,-0.405851,0.813787,-1.250857
10078,1.0,1.565182,1.46385,1.006993,1.224502,0.413919,0.337951,0.000000,0.915923,0.117912,0.004641,0.904534,1.233630,0.265307,0.945249,-0.510371


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X_, data.iloc[:, -1], test_size=0.2, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8064, 16), (2016, 16), (8064,), (2016,))

In [7]:
acc = []
r2score = []

for i in range(10, 16):

    features_selector_forward = sfs(  LogisticRegression(multi_class='multinomial'),
                k_features=i, 
                forward=True, 
                verbose=0, 
                scoring='neg_mean_squared_error', cv=5, n_jobs=-1 )

    features_selector_forward.fit(x_train, y_train)

    X = X_[list(features_selector_forward.k_feature_names_)]

    X_train, X_test, Y_train, Y_test = train_test_split(X, data.iloc[:, -1], test_size=0.2, random_state=0)

    clf = LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1).fit(X_train, Y_train)

    y_pred = clf.predict(X_test)
    
    acc.append(accuracy_score(Y_test, y_pred))
    r2score.append(r2_score(Y_test, y_pred))

    print("For {} features, score = {}".format(i, accuracy_score(Y_test, y_pred)))
    print("For {} features, R2score = {}".format(i, r2_score(Y_test, y_pred)))
    print()
    
print('- - - - - - - - - - - -')
print("Mean Acc = ", (sum(acc) / len(acc)))
print("Mean R2 = ", (sum(r2score) / len(r2score)))

For 10 features, score = 0.8030753968253969
For 10 features, R2score = 0.4754037420047351

For 11 features, score = 0.8005952380952381
For 11 features, R2score = 0.4793850528913063

For 12 features, score = 0.8015873015873016
For 12 features, R2score = 0.48523992184214626

For 13 features, score = 0.8035714285714286
For 13 features, R2score = 0.48828445369658313

For 14 features, score = 0.8025793650793651
For 14 features, R2score = 0.48781606418051593

For 15 features, score = 0.8070436507936508
For 15 features, R2score = 0.44027452829969504

- - - - - - - - - - - -
Mean Acc =  0.8030753968253971
Mean R2 =  0.4760672938191637


In [8]:
features_selector_forward = sfs(  LogisticRegression(multi_class='multinomial'),
            k_features=13, 
            forward=True, 
            verbose=0, 
            scoring='neg_mean_squared_error', cv=5, n_jobs=-1 )

features_selector_forward.fit(x_train, y_train)

SequentialFeatureSelector(estimator=LogisticRegression(multi_class='multinomial'),
                          k_features=(13, 13), n_jobs=-1,
                          scoring='neg_mean_squared_error')

In [75]:
features_selector_forward.k_feature_names_

('Gender',
 'Age',
 'Height(cm)',
 'Weight(kg)',
 'Exercise',
 'Target',
 'How motivated are you for being healthy\n(in %)',
 'Body Mass Index (BMI)',
 'Basal Metabolic Rate (BMR)',
 'Type',
 'Body Fat Percentage (BFP)',
 'Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie)',
 'Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie)')

In [9]:
x_train_f = x_train[list(features_selector_forward.k_feature_names_)]
x_train_f.shape

(8064, 13)

In [35]:
log_reg_forward = sm.MNLogit(y_train.values.ravel(), x_train_f).fit(method='bfgs', maxiter=1000)

Optimization terminated successfully.
         Current function value: 1.039078
         Iterations: 307
         Function evaluations: 308
         Gradient evaluations: 308


In [36]:
log_reg_forward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                 8064
Model:                        MNLogit   Df Residuals:                     7999
Method:                           MLE   Df Model:                           60
Date:                Fri, 09 Dec 2022   Pseudo R-squ.:                  0.3096
Time:                        13:55:23   Log-Likelihood:                -8379.1
converged:                       True   LL-Null:                       -12136.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================================================================
                                                                            y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------
Gender                                                                             -0.0519      0.294     -0.177      0.860      -0.628       0.524
Age                                                                                -0.3146      0.212     -1.487      0.137      -0.729       0.100
Height(cm)                                                                          1.9787      0.226      8.760      0.000       1.536       2.421
Weight(kg)                                                                         -4.6805      0.558     -8.385      0.000      -5.775      -3.586
Exercise                                                                            0.9935      0.342      2.901      0.004       0.322       1.665
Target                                                                              1.6606      0.059     27.913      0.000       1.544       1.777
How motivated are you for being healthy
(in %)                                      0.1351      0.047      2.896      0.004       0.044       0.227
Body Mass Index (BMI)                                                               2.6148      0.688      3.802      0.000       1.267       3.963
Basal Metabolic Rate (BMR)                                                          1.3888      0.539      2.576      0.010       0.332       2.446
Type                                                                                0.0433      0.078      0.555      0.579      -0.110       0.196
Body Fat Percentage (BFP)                                                           0.5803      0.463      1.253      0.210      -0.327       1.488
Daily Calorie Need (Based on BMR and Exercise)
(in Calorie)                        -1.8802      0.578     -3.252      0.001      -3.013      -0.747
Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)
(in Calorie)     0.0014      0.069      0.020      0.984      -0.134       0.137
---------------------------------------------------------------------------------------------------------------------------------------------------
                                                                            y=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------
Gender                                                                              0.6003      0.340      1.764      0.078      -0.067       1.267
Age                                                                                 0.4948      0.225      2.197      0.028       0.053       0.936
Height(cm)                                                                          0.6858      0.234      2.929      0.003       0.2

In [12]:
log_reg_forward.aic

16888.22697390568

In [76]:
log_reg_forward.bic

17342.934999975347

## Resampling Methods
    - Using Cross validation

In [13]:
from sklearn.model_selection import cross_validate

lg_resultant = cross_validate(LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1), 
                              x_train_f, 
                              y_train, 
                              cv=5,
                              return_train_score=True)

print("Mean Training Score = ", lg_resultant['train_score'].mean()*100)
print("Mean Testing Score = ", lg_resultant['test_score'].mean()*100)

Mean Training Score =  81.26240129824114
Mean Testing Score =  80.77869943188026


    - Using Bootstrap

In [74]:
train_score = []
test_score = []
n_times = 10

## Performing bootstrapping
for i in range(n_times):
    
    x_train, x_test, y_train, y_test = train_test_split(X_[list(features_selector_forward.k_feature_names_)], data.iloc[:, -1], test_size=0.2, random_state=42+i)
    
    #choose from different tunable hyper parameters
    clf = LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1).fit(x_train, y_train)
    y_pred = clf.predict(x_test)
 
    #Measuring accuracy on Testing Data
    Accuracy = accuracy_score(y_test, y_pred)
    
    # Storing accuracy values
    test_score.append(Accuracy)
    train_score.append(accuracy_score(y_train, clf.predict(x_train)))
    
################################################
# Result of all bootstrapping trials
print("Mean Training Score = ", np.mean(train_score)*100)
print("Mean Testing Score = ", np.mean(test_score)*100)

Mean Training Score =  81.06150793650792
Mean Testing Score =  80.80853174603175


### Applying GAM for classification